In [12]:
import json as js
from tqdm import tqdm
import string
import sys
import keras
import tensorflow as tf
import json
import numpy as np
import dask.array as da

import os
os.environ["CUDA_DEVICE_ORDER"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from rnn_bimodal import RNNBimodal
from text_processing import textProcessing
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer as Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [13]:
if "all_descriptions_train.zarr" in os.listdir('.') and "embedding_train_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_train = da.from_zarr("embedding_train_128.zarr")
        desc_train = da.from_zarr("all_descriptions_train.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

if "all_descriptions_validation.zarr" in os.listdir('.') and "embedding_validation_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_validation = da.from_zarr("embedding_validation_128.zarr")
        desc_validation = da.from_zarr("all_descriptions_validation.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

if "all_descriptions_test.zarr" in os.listdir('.') and "embedding_test_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_test = da.from_zarr("embedding_test_128.zarr")
        desc_test = da.from_zarr("all_descriptions_test.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

In [14]:
desc_train

dask.array<from-zarr, shape=(5000,), dtype=<U177, chunksize=(827,), chunktype=numpy.ndarray>

In [15]:
allText = np.append(desc_train.compute(),desc_validation.compute())
allText = np.append(allText,desc_test.compute())
tokenizer = Tokenizer()
tokenizer.fit_on_texts(allText)
vocab_size = len(tokenizer.word_index) + 1

In [16]:
max_length = max(len(d.split()) for d in allText)
max_length

32

In [17]:
def count_length(tokenizer, descriptions):
    Y = 0
    for i in range(len(descriptions)):
        seq = tokenizer.texts_to_sequences([(descriptions[i].compute()).tolist()])[0]
        Y+=len(seq)-1
    return Y

In [18]:
if 'best_model_rnn.h5' in os.listdir():
    model = load_model('best_model_rnn.h5')
else:
    model = RNNBimodel.build(vocab_size, max_length)

print(model.summary())

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 32, 256)      1081088     input_16[0][0]                   
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 80)           0           input_15[0][0]                   
____________________________________________________________________________________________

In [19]:
epochs = 50
nb_train_samples = count_length(tokenizer, desc_train)
nb_validation_samples = count_length(tokenizer, desc_validation)
print(nb_train_samples, nb_validation_samples)

48255 19277


In [20]:
with tf.device('/gpu:0'):
    stop_early = EarlyStopping(monitor='val_loss',patience = 10)
    ModelCheck = ModelCheckpoint('best_model_rnn.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    callbacks = [stop_early, ModelCheck]
    for i in range(epochs):
        train_generator = textProcessing.data_generator(desc_train, embedding_train, tokenizer, max_length, vocab_size)
        validation_generator = textProcessing.data_generator(desc_validation, embedding_validation, tokenizer, max_length, vocab_size)
        i_train, o_train, batch_size_train = next(train_generator)
        i_validation, o_validation, batch_size_validation = next(validation_generator)
        model.fit(
            x = i_train, 
            y = o_train, 
            epochs=1, 
            steps_per_epoch=nb_train_samples//batch_size_train, 
            verbose=1, 
            validation_data=(i_validation, o_validation), 
            validation_steps=nb_validation_samples//batch_size_validation,
            callbacks=callbacks)

  39/5361 [..............................] - ETA: 11:42 - loss: 2.4114e-08

KeyboardInterrupt: 

In [ ]:
model = load_model('best_model_rnn.h5')

textProcessing.generate_desc(model, tokenizer, embedding_test[0].compute(), max_length)
# embedding_test[0].compute().shape